<a href="https://colab.research.google.com/github/ozturkergin/ozturkergin/blob/main/TEFAS_PowerBI_Import_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install bs4 --quiet

In [2]:
pip install urllib3 --quiet

In [3]:
pip install marshmallow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.7 MB/s eta 0:00:00


In [4]:
!pip install pandas_ta==0.3.14b --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
pip install free-proxy --quiet

  Preparing metadata (setup.py) ... done


In [6]:
import requests
import pandas as pd
import pandas_ta as ta
import math
import ssl
import concurrent.futures
import time

from datetime import datetime, timedelta, date
from typing import Dict, List, Optional, Union
from marshmallow import Schema, fields, EXCLUDE, pre_load, post_load
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
from urllib3.poolmanager import PoolManager
from urllib3.util.retry import Retry
from fp.fp import FreeProxy

# Special thanks to https://github.com/burakyilmaz321

class InfoSchema(Schema):
    code = fields.String(data_key="FONKODU", allow_none=True)
    fonunvantip = fields.String(data_key="FONUNVANTIP", allow_none=True)
    date = fields.Date(data_key="TARIH", allow_none=True)
    price = fields.Float(data_key="FIYAT", allow_none=True)
    title = fields.String(data_key="FONUNVAN", allow_none=True)
    market_cap = fields.Float(data_key="PORTFOYBUYUKLUK", allow_none=True)
    number_of_shares = fields.Float(data_key="TEDPAYSAYISI", allow_none=True)
    number_of_investors = fields.Float(data_key="KISISAYISI", allow_none=True)

    @pre_load
    def pre_load_hook(self, input_data, **kwargs):
        # Convert milliseconds Unix timestamp to date
        seconds_timestamp = int(input_data["TARIH"]) / 1000
        input_data["TARIH"] = date.fromtimestamp(seconds_timestamp).isoformat()
        return input_data

    @post_load
    def post_load_hool(self, output_data, **kwargs):
        # Fill missing fields with default None
        output_data = {f: output_data.setdefault(f) for f in self.fields}
        return output_data

    class Meta:
        unknown = EXCLUDE

class tefas_get:

    root_url = "https://www.tefas.gov.tr"
    #detail_endpoint = "/api/DB/BindHistoryAllocation"
    info_endpoint = "/api/DB/BindHistoryInfo"

    @staticmethod
    def get_combobox_items(url, select_id):
        # Send a GET request to the URL
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code != 200:
            raise Exception(f"Failed to fetch the URL: {response.status_code}")

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the select element by its ID
        select_element = soup.find('select', id=select_id)

        # Check if the select element exists
        if not select_element:
            raise Exception(f"Select element with id '{select_id}' not found")

        # Get all option elements within the select element
        options = select_element.find_all('option')
        options = list(filter(None, options))

        # Extract the text and value for each option
        items = []
        for option in options:
            value = option.get('value')
            items.append(value)

        items.remove('')
        #items.remove('Serbest')

        return items

    def fetch_info( self, fonunvantip, start_date_initial, end_date_initial ):

        counter = 1
        start_date = start_date_initial
        end_date = end_date_initial
        range_date = end_date_initial - start_date_initial
        range_interval = 90
        info_schema = InfoSchema(many=True)
        info_result = pd.DataFrame()

        if range_date.days > range_interval :
           counter = range_date.days / range_interval
           counter = math.ceil(counter)
           end_date = start_date + timedelta(days=range_interval)

        while counter > 0:
           counter -= 1

           data = {
                    "fontip": "YAT",
                    "bastarih": self._parse_date(start_date),
                    "bittarih": self._parse_date(end_date),
                    "fonunvantip": fonunvantip,
                    "fonkod": "",
                  }

           info = self._do_post(self.info_endpoint, data)
           info = info_schema.load(info)
           info = pd.DataFrame(info, columns=info_schema.fields.keys())
           info['fonunvantip'] = fonunvantip
           info = info[info['price'] != 0]
           info_result = pd.concat([info_result, info])
           info_result = info_result.reset_index(drop=True)
           info = info.reset_index(drop=True)

           if counter > 0 :
              start_date = end_date + timedelta(days=1)
              end_date = end_date + timedelta(days=range_interval)
              if end_date > end_date_initial :
                 end_date = end_date_initial

        return info_result

    def fetch_info_serial( self, fonunvantips, start_date_initial, end_date_initial ):
        merged = pd.DataFrame()

        for fonunvantip in fonunvantips:
            info = self.fetch_info(fonunvantip, start_date_initial, end_date_initial)
            merged = pd.concat([merged, info])
            print(f"{fonunvantip} - {len(info)} records added total records: {len(merged)} " )

        return merged

    def fetch_info_concurrently( self, fonunvantips, start_date_initial, end_date_initial ):
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            merged = pd.DataFrame()
            # Submit all tasks to the executor
            futures = {executor.submit(self.fetch_info, fonunvantip, start_date_initial, end_date_initial): fonunvantip for fonunvantip in fonunvantips}

            # Retrieve results as they complete
            for future in concurrent.futures.as_completed(futures):
                info = future.result()
                merged = pd.concat([merged, info])
                print(f"{fonunvantip} - {len(info)} records added total records: {len(merged)} " )

            return merged

    def fetch(
        self,
        start: Union[str, datetime],
        end: Optional[Union[str, datetime]] = None,
        columns: Optional[List[str]] = None,
		concurrent = False
    ) -> pd.DataFrame:

        start_date_initial = datetime.strptime(start, "%Y-%m-%d")
        end_date_initial = datetime.strptime(end or start, "%Y-%m-%d")

        merged = pd.DataFrame()
        fonunvantips = self.get_combobox_items(url="https://www.tefas.gov.tr/TarihselVeriler.aspx", select_id="DropDownListFundTypeExplanationYAT")

        if concurrent != False :
            merged = self.fetch_info_concurrently(fonunvantips, start_date_initial, end_date_initial)
        else :
            merged = self.fetch_info_serial(fonunvantips, start_date_initial, end_date_initial)

        merged = merged[columns] if columns and not merged.empty else merged

        return merged

    def _do_post(self, endpoint: str, data: Dict[str, str]) -> Dict[str, str]:
        #time.sleep(1)
        session = requests.Session()
        retry_strategy = Retry(
            total=3,  # Total number of retries
            backoff_factor=1,  # Backoff factor for retries
            status_forcelist=[429, 500, 502, 503, 504],  # Retry on these status codes
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        session.mount("https://", adapter)

        timestamp = int(time.time() * 1000)  # Get current timestamp in milliseconds

        headers = {
        "Connection": "keep-alive",
        "Cache-Control": "no-cache",
        "Pragma": "no-cache",
        "X-Requested-With": "XMLHttpRequest",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Accept-Language": "tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Origin": "https://www.tefas.gov.tr",
        "Referer": "https://www.tefas.gov.tr/TarihselVeriler.aspx?timestamp={timestamp}" ,
        }

        proxy_address = FreeProxy(country_id=['US', 'BR'], timeout=0.3, rand=True).get() # Get proxy address as a string
        proxies = { # Create a proxies dictionary
            'http': proxy_address,
            'https': proxy_address
        }
        response = session.post(
            url=f"{self.root_url}/{endpoint}",
            data=data,
            proxies=proxies,
            headers=headers,
        )
        # Check the response status code and content
        if response.status_code != 200:
            print(f"Request failed with status code: {response.status_code}")
            print(f"Response content: {response.text}")
            return {}  # Return an empty dictionary if the request failed
        try:
            return response.json().get("data", {})
        except ValueError as e:
            print(f"Error decoding JSON response: {e}")
            print(f"Response content: {response.text}")
            return {}

    def _parse_date(self, date: Union[str, datetime]) -> str:
        if isinstance(date, datetime):
            formatted = datetime.strftime(date, "%d.%m.%Y")
        elif isinstance(date, str):
            try:
                parsed = datetime.strptime(date, "%Y-%m-%d")
            except ValueError as exc:
                raise ValueError(
                    "Date string format is incorrect. " "It should be `YYYY-MM-DD`"
                ) from exc
            else:
                formatted = datetime.strftime(parsed, "%d.%m.%Y")
        else:
            raise ValueError(
                "`date` should be a string like 'YYYY-MM-DD' "
                "or a `datetime.datetime` object."
            )
        return formatted

tefas = tefas_get()

time_delta = 366
start_date_calc = date.today() - timedelta(days=time_delta)
date_start = start_date_calc.strftime("%Y-%m-%d")
date_end = date.today().strftime("%Y-%m-%d")

fetched_data = pd.DataFrame()
fetched_data = tefas.fetch(start=date_start, end=date_end, columns=["code", "date", "price", "fonunvantip", "title", "market_cap", "number_of_shares", "number_of_investors" ], concurrent=True)
fetched_data['date'] = pd.to_datetime(fetched_data['date'], errors='coerce')
fetched_data['date'].dt.strftime('%Y-%m-%d')
fetched_data['date'] = fetched_data['date'].dt.date
fetched_data['price'].astype(float,False)
fetched_data.rename(columns={'price': 'close'}, inplace=True)
fetched_data['market_cap'].astype(float,False)
fetched_data['number_of_shares'].astype(float,False)
fetched_data['number_of_investors'].astype(float,False)
fetched_data[(fetched_data!=0)&(pd.isnull(fetched_data))]

fetched_data_agg = fetched_data.copy()
fetched_data_agg = fetched_data_agg.groupby(['code', 'title', 'fonunvantip'])['close'].mean().reset_index()
fon_table = fetched_data_agg.pivot(index=['code','title'], columns='fonunvantip', values='close').notnull()
del fetched_data_agg

fetched_data = fetched_data.drop(['fonunvantip','title'], axis=1)
fetched_data = fetched_data.drop_duplicates(subset=['code','date'], keep='first')


AttributeError: 'str' object has no attribute 'get'

In [16]:
def calculate_rsi(group, window_length):
    rsi_values = ta.rsi(close=group['close'], length=window_length)
    group['rsi'] = rsi_values  # Assign RSI values directly
    return group
fetched_data

code        date      close    market_cap  number_of_shares  \
code                                                                     
AAK  25     AAK  2024-06-20  22.315576  1.205506e+08         5402082.0   
     24     AAK  2024-06-21  22.701188  1.266432e+08         5578702.0   
     23     AAK  2024-06-24  22.703596  1.270491e+08         5595991.0   
     22     AAK  2024-06-25  22.660706  1.267555e+08         5593627.0   
     21     AAK  2024-06-26  22.649760  1.270949e+08         5611312.0   
...         ...         ...        ...           ...               ...   
ZZL  38830  ZZL  2024-07-22  45.722298  2.328723e+07          509319.0   
     38829  ZZL  2024-07-23  45.634717  2.324263e+07          509319.0   
     38828  ZZL  2024-07-24  45.605468  2.322773e+07          509319.0   
     38827  ZZL  2024-07-25  45.547069  2.319799e+07          509319.0   
     38826  ZZL  2024-07-26  45.854076  2.335435e+07          509319.0   

            number_of_investors  rsi  
code                                  
AAK  25                  1093.0    0  
     24                  1103.0    0  
     23                  1112.0    0  
     22                  1120.0    0  
     21                  1125.0    0  
...                         ...  ...  
ZZL  38830                 15.0    0  
     38829                 15.0    0  
     38828                 15.0    0  
     38827                 15.0    0  
     38826                 15.0    0  

[38852 rows x 7 columns]

In [26]:
# Group by stock and apply the RSI calculation
window_length = 14
fetched_data = fetched_data.reset_index(drop=True)
fetched_data.sort_values(by=['code', 'date'], inplace=True)
fetched_data = fetched_data.groupby(['code']).apply(calculate_rsi, window_length=window_length)
fetched_data = fetched_data.fillna(0)
fetched_data['rsi'].astype(float,False)
fetched_data.to_excel("fetched_data.xlsx", index=False)

def calculate_rsi(group, window_length):
    rsi_values = ta.rsi(close=group['close'], length=window_length)
    group['rsi'] = rsi_values  # Assign RSI values directly
    return group

In [23]:
fon_table

,fonunvantip,Agresif Değişken,Alternatif,Altın,Borçlanma Araçları,Dengeli Değişken,Değişken,Diğer Fon Sepeti,Döviz,Döviz Cinsinden İhraç (Dolar),Döviz Cinsinden İhraç (Euro),...,Sektör,Serbest,Sürdürülebilirlik Fonları,Uzun Vadeli,Yabancı,Yabancı Fon Sepeti,Çalışanlarına Yönelik,Çoklu Varlık,Özel,İştirak
code,title,,,,,,,,,,,,,,,,,,,,,
AAK,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
AAL,ATA PORTFÖY PARA PİYASASI (TL) FONU,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
AAS,ATA PORTFÖY FON SEPETİ SERBEST FONU,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİSSE SENEDİ YOĞUN FON),False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABG,ATLAS PORTFÖY DÖRDÜNCÜ SERBEST (TL) FON,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZVB,ATLAS PORTFÖY SERBEST (TL) FON,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
ZVO,ZİRAAT PORTFÖY ÜÇÜNCÜ SERBEST (TL) FON,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
ZYC,ZİRAAT PORTFÖY DÖRDÜNCÜ SERBEST (TL) FON,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [25]:
fetched_data

code        date      close    market_cap  number_of_shares  \
code                                                                     
AAK  0      AAK  2024-06-20  22.315576  1.205506e+08         5402082.0   
     1      AAK  2024-06-21  22.701188  1.266432e+08         5578702.0   
     2      AAK  2024-06-24  22.703596  1.270491e+08         5595991.0   
     3      AAK  2024-06-25  22.660706  1.267555e+08         5593627.0   
     4      AAK  2024-06-26  22.649760  1.270949e+08         5611312.0   
...         ...         ...        ...           ...               ...   
ZZL  38847  ZZL  2024-07-22  45.722298  2.328723e+07          509319.0   
     38848  ZZL  2024-07-23  45.634717  2.324263e+07          509319.0   
     38849  ZZL  2024-07-24  45.605468  2.322773e+07          509319.0   
     38850  ZZL  2024-07-25  45.547069  2.319799e+07          509319.0   
     38851  ZZL  2024-07-26  45.854076  2.335435e+07          509319.0   

            number_of_investors        rsi  
code                                        
AAK  0                   1093.0   0.000000  
     1                   1103.0   0.000000  
     2                   1112.0   0.000000  
     3                   1120.0   0.000000  
     4                   1125.0   0.000000  
...                         ...        ...  
ZZL  38847                 15.0  88.023282  
     38848                 15.0  83.233915  
     38849                 15.0  81.636381  
     38850                 15.0  78.400821  
     38851                 15.0  82.359157  

[38852 rows x 7 columns]

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/data_table.py", line 192, in _repr_mimebundle_
    dataframe = self._preprocess_dataframe()
  File "/usr/local/lib/python3.10/dist-packages/google/colab/data_table.py", line 180, in _preprocess_dataframe
    dataframe = dataframe.reset_index()
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py", line 6209, in reset_index
    new_obj.insert(
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py", line 4772, in insert
    raise ValueError(f"cannot insert {column}, already exists")
ValueError: cannot insert code, already exists
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/data_table.py", line 204, in _repr_javascript_module_
    return self._gen_js(self._preprocess_dataframe())
  File "/usr/local/lib/python3.10/dist-packages/google/colab/data_table.py", line 180, in _preprocess_dataframe
    dataframe = datafram